In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import regex as re

In [2]:
def file_to_sentence_list(file_path):
    with open(file_path, 'r') as file:
        text = file.read()

    sentences = [sentence.strip() for sentence in re.split(
        r'(?<=[.!?])\s+', text) if sentence.strip()]

    return sentences

file_path = '/kaggle/input/dataset1/disk1.txt'
text_data = file_to_sentence_list(file_path)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1

# Create input sequences
input_sequences = []
for line in text_data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(
    input_sequences, maxlen=max_sequence_len, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]

y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [3]:
model = Sequential()
model.add(Embedding(total_words, 10,
                    input_length=max_sequence_len-1))
model.add(LSTM(128))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [4]:
# Train
model.fit(X, y, epochs=50, verbose=1)
model.save('model.h5')

Epoch 1/50
4191/4191 ━━━━━━━━━━━━━━━━━━━━ 528s 125ms/step - accuracy: 0.0342 - loss: 7.5810
Epoch 2/50
4191/4191 ━━━━━━━━━━━━━━━━━━━━ 546s 130ms/step - accuracy: 0.0521 - loss: 6.9089
Epoch 3/50
4191/4191 ━━━━━━━━━━━━━━━━━━━━ 549s 131ms/step - accuracy: 0.0698 - loss: 6.5445
Epoch 4/50
4191/4191 ━━━━━━━━━━━━━━━━━━━━ 545s 130ms/step - accuracy: 0.0876 - loss: 6.2484
Epoch 5/50
4191/4191 ━━━━━━━━━━━━━━━━━━━━ 554s 132ms/step - accuracy: 0.1007 - loss: 5.9548
Epoch 6/50
4191/4191 ━━━━━━━━━━━━━━━━━━━━ 545s 128ms/step - accuracy: 0.1076 - loss: 5.6832
Epoch 7/50
4191/4191 ━━━━━━━━━━━━━━━━━━━━ 561s 128ms/step - accuracy: 0.1189 - loss: 5.4079
Epoch 8/50
4191/4191 ━━━━━━━━━━━━━━━━━━━━ 567s 129ms/step - accuracy: 0.1328 - loss: 5.1468
Epoch 9/50
4191/4191 ━━━━━━━━━━━━━━━━━━━━ 547s 130ms/step - accuracy: 0.1524 - loss: 4.9039
Epoch 10/50
4191/4191 ━━━━━━━━━━━━━━━━━━━━ 538s 128ms/step - accuracy: 0.1734 - loss: 4.6805
Epoch 11/50
4191/4191 ━━━━━━━━━━━━━━━━━━━━ 538s 128ms/step - accuracy: 0.1939 -

In [5]:
# Generate next word predictions
seed_text = "what"
next_words = 5

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences(
        [token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted_probs = model.predict(token_list)
    predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
    seed_text += " " + predicted_word

print("Next predicted words:", seed_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Next predicted words: what do you think about you
